In [18]:
import os
from csv import DictWriter
import sys
import local_utils
import pandas as pd
import time
import requests
try:
     from PIL import Image
except ImportError:
     import Image

import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from local_utils import detect_lp
from os.path import splitext,basename
from keras.models import model_from_json
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
import numpy as np
face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [32]:
df = pd.DataFrame(columns = ['date', 'v_number']) 
df.to_csv('data_new.csv')
try:
    os.mkdir("faces")
except FileExistsError:
    print("already exists")
    pass
try:
    os.mkdir("plates")
except FileExistsError:
    print("already exists")
    pass

already exists


In [20]:
def csv_updater(v_number):    
        with open('data_new.csv','a', newline = '') as f:
            csv_file = DictWriter(f, fieldnames = ["date","v_number"])
            csv_file.writerows([
            {'date': [time.asctime( time.localtime(time.time()) )], 'v_number': v_number}
                ])
        f.close()


In [21]:
def most_frequent(List): 
    dict = {} 
    count, itm = 0, '' 
    for item in reversed(List): 
        dict[item] = dict.get(item, 0) + 1
        if dict[item] >= count : 
            count, itm = dict[item], item 
    return(itm) 

In [25]:
def detect(frame):
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) #changing RGB to gray for better classification
    faces = face_classifier.detectMultiScale(gray,1.3,5)
    if faces is not ():
        for (x,y,w,h) in faces:
            cv2.rectangle(frame ,(x-w//2,y-h//2), (x+2*w ,y+2*h),(255,0,0),2)
            img = frame[y-h//2:y+2*h , x-w//2:x+2*w]
        return(img) 
    else:
        return(frame)
def save_img(img):
    name = str(time.asctime())[4:16]
    cv2.imwrite("faces/"+name.jpg , img )

In [23]:
def load_model(path):
    try:
        path = splitext(path)[0]
        with open('%s.json' % path, 'r') as json_file:
            model_json = json_file.read()
        model = model_from_json(model_json, custom_objects={})
        model.load_weights('%s.h5' % path)
        print("Loading model successfully...")
        return model
    except Exception as e:
        print(e)

In [8]:
def preprocess_image(img,resize=True):
    #img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255
    if resize:
        img = cv2.resize(img, (224,224))
    return img

In [9]:
def get_plate(img, Dmax=608, Dmin=256):
    vehicle = preprocess_image(img)
    ratio = float(max(vehicle.shape[:2])) / min(vehicle.shape[:2])
    side = int(ratio * Dmin)
    bound_dim = min(side, Dmax)
    _ , LpImg, _, cor = detect_lp(wpod_net, vehicle, bound_dim, lp_threshold=0.5)
    return(vehicle, LpImg, cor)


In [10]:
def image_for_ocr(LpImg):
    if (len(LpImg)): #check if there is at least one license image
        # Scales, calculates absolute values, and converts the result to 8-bit.
        plate_image = cv2.convertScaleAbs(LpImg[0], alpha=(255.0))
        plt.imshow(plate_image)
        plt.title("Plate image")
        # convert to grayscale and blur the image
        gray = cv2.cvtColor(plate_image, cv2.COLOR_BGR2GRAY)
        #blur = cv2.GaussianBlur(gray,(7,7),0)
        
        # Applied inversed thresh_binary 
        binary = cv2.threshold(gray, 180, 255,
                            cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        binary= cv2.bitwise_not(binary)
        kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        thre_mor = cv2.morphologyEx(binary, cv2.MORPH_DILATE, kernel3)
    return([plate_image,gray,binary,kernel3,thre_mor])

In [17]:
def find_num(img):
    #img = cv2.imread(path)
    LpImg =0
    try:
        vehicle, LpImg,cor = get_plate(img)
    except AssertionError:
        pass
    s=[]
    if LpImg:
        ocr_image=image_for_ocr(LpImg)[0]
        number= pytesseract.image_to_string(ocr_image,lang="eng")
        for i in number:
            if i.isalnum() and (not i.islower()):
                s.append(i)
        return("".join(s),LpImg)
    else:
        return(0)

In [12]:
def draw_box(image_path, cor, thickness=3): 
    pts=[]  
    x_coordinates=cor[0][0]
    y_coordinates=cor[0][1]
    # store the top-left, top-right, bottom-left, bottom-right 
    # of the plate license respectively
    for i in range(4):
        pts.append([int(x_coordinates[i]),int(y_coordinates[i])])
    
    pts = np.array(pts, np.int32)
    pts = pts.reshape((-1,1,2))
    vehicle_image = preprocess_image(image_path)
    
    cv2.polylines(vehicle_image,[pts],True,(0,255,0),thickness)
    return vehicle_image

In [13]:
wpod_net_path = "wpod-net.json"
wpod_net = load_model(wpod_net_path)

Loading model successfully...


In [14]:
# cap = cv2.VideoCapture("test.mp4") #path to video
# num=[]
# while cap.isOpened():
#     time.sleep(1)
#     ret,frame = cap.read()
#     if ret:
#         gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
#         number = find_num(gray)
#         if number:
#             num.append(number)
#         else:
#             continue
#         if num[0] != num[len(num)-1] and len(num)>1:
#             print(num)
#             csv_updater(num[0])
#             num = []
#         else:
#             continue
#         if cv2.waitKey(1) & 0xFF == ord('q'):  #13 is enter key
#             break
#     else:
#         break
# cap.release()

In [1]:
cap_plate = cv2.VideoCapture(0)
num=[]
while cap_plate.isOpened():
    time.sleep(0.05)
    ret,frame = cap_plate.read()
    if ret:
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        try:
            vehicle, LpImg,cor = get_plate(frame)
            plate = draw_box(frame,cor)
        except AssertionError:
            plate = frame
        cv2.imshow("plate",plate)
        if cv2.waitKey(1) & 0xFF == ord('q'):  #13 is enter key
            break        
    else:
        break
cap_plate.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [15]:
cap_plate = cv2.VideoCapture(0) #path to video
num=[]
while cap_plate.isOpened():
    time.sleep(1)
    ret,frame = cap.read()
    if ret:
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        number = find_num(gray)
        if number:
            num.append(number)
        else:
            continue
        #if num[0] != num[len(num)-1] and len(num)>1:
        if len(num)>20:
            n=most_frequent(num)
            csv_updater(n)
            print(n)
            num = []
        else:
            continue
        if cv2.waitKey(1) & 0xFF == ord('q'):  #13 is enter key
            break
    else:
        break
cap.release()

In [30]:
cap_face = cv2.VideoCapture(0)
while cap_face.isOpened():
    #time.sleep(0.2)
    ret,frame = cap_face.read()
    if ret:
        face_image = detect(frame)
        cv2.imshow("video" , face_image)
        if cv2.waitKey(1) & 0xFF == ord('q'): # press enter and "q" key to close the window
            break
    else:
        break
cap_face.release()
cv2.destroyAllWindows()

In [ ]:
#number plate csv updater
def csv_updater(v_number):
    name = time.asctime()[4:16]
    name = name.replace(":", "_")
    with open('data_new.csv','a', newline = '') as f:
        csv_file = DictWriter(f, fieldnames = ["date","v_number",'plate_path','face_path'])
        csv_file.writerows([
        {'date': [time.asctime( time.localtime(time.time()) )], 'v_number': v_number,
         'plate_path':"plates/"+v_number+ ".jpg",'face_path':"faces/"+name+".jpg"}
            ])
    f.close()


# plate path updater        
#def plate_updater():    
#        with open('data_new.csv','a', newline = '') as f:
#            csv_file = DictWriter(f, fieldnames = ["date","v_number"])
#            csv_file.writerows([
#            {'date': [time.asctime( time.localtime(time.time()) )], 'v_number': v_number}
#                ])
#        f.close()